<a href="https://colab.research.google.com/github/fabian-andesite/corma/blob/main/BrechaSalarialForestalOficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
# from ydata_profiling import ProfileReport
import numpy as np
import pydrive

# Load Data

##Option 2: Manually upload excel datafile to Colab Files. Loading files aferwards with pandas


In [17]:
data_path = "base-de-datos-concursantes.xlsx"
tab_name = "DATOS"
data = pd.read_excel(data_path, sheet_name=tab_name)
data.describe()


,ID_PERSONA,edad,sexo,ecivil,e6a,e8,oficio4_08,o10,o15,o18,...,s7,s13,s13_fonasa,s16,s28,r1a_esp_cod,r3,os1,genero,trans
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,210.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,119.000000,1000.000000,873.000000,1000.000000,1000.000000,66.000000,1000.000000,451.000000,451.000000,6.000000
mean,500.500000,43.361000,1.125000,3.606000,9.579000,2.614286,6806.259000,43.830000,4.960000,-1.546000,...,3.915966,0.610000,-5.372279,3.979000,17.859000,480.909091,9.998000,-2.917960,0.740576,1.833333
std,288.819436,13.376233,0.330884,3.027859,2.190561,6.496137,2154.881838,12.359282,0.229025,16.070163,...,0.402952,6.901705,25.911458,7.240095,11.523711,54.198986,2.449898,19.375928,6.677433,0.408248
min,1.000000,16.000000,1.000000,1.000000,1.000000,-88.000000,1211.000000,-88.000000,3.000000,-99.000000,...,2.000000,-88.000000,-88.000000,-88.000000,-88.000000,214.000000,1.000000,-99.000000,-99.000000,1.000000
25%,250.750000,32.000000,1.000000,1.000000,8.000000,2.000000,5938.750000,45.000000,5.000000,1.000000,...,4.000000,1.000000,2.000000,5.000000,22.000000,414.750000,11.000000,1.000000,1.000000,2.000000
50%,500.500000,43.000000,1.000000,2.000000,9.000000,2.000000,7522.000000,45.000000,5.000000,1.000000,...,4.000000,1.000000,3.000000,5.000000,22.000000,509.000000,11.000000,1.000000,1.000000,2.000000
75%,750.250000,54.000000,1.000000,8.000000,11.000000,4.000000,8322.000000,45.000000,5.000000,1.000000,...,4.000000,1.000000,3.000000,5.000000,22.000000,513.000000,11.000000,1.000000,1.000000,2.000000
max,1000.000000,79.000000,2.000000,8.000000,14.000000,7.000000,9624.000000,84.000000,5.000000,3.000000,...,4.000000,4.000000,4.000000,5.000000,22.000000,513.000000,11.000000,4.000000,5.000000,2.000000


# Data Cleaning
Deletes column with too many nan and drop the rows with nan




##Discard columns


In [5]:
# decides if discard a column or not
def discard_column(df, variable, rate_limit=0.8):
  # criteria 1: if nan is above 80% then, the variable is discarded

  nan_rate = (df[variable].isna().sum())/len(df)
  if nan_rate > rate_limit:
    return True
  else:
    return False


### Select variables according to numerical criteria (example % of NaN)

In [6]:
def clean_data(df, rate_limit=0.8):

  clean_data = data.replace([-88, -99], np.nan )

  # filter columns
  relevant_variables = list(clean_data.columns)
  ncolumns_deleted = 0

  for col in clean_data.columns:
    if discard_column(clean_data, col, rate_limit):
      ncolumns_deleted+=1
      relevant_variables.remove(col)

  print("# of columns deleted: {0}".format(ncolumns_deleted))
  clean_data = clean_data[relevant_variables]
  clean_data.head()

  df_sin_nan = clean_data.dropna()
  return df_sin_nan




### Apply cleaning function to database

In [7]:
# this clean de database deleting all columns that have more 5% of NaN
result = clean_data(data, 0.05)
result.describe()

# of columns deleted: 13


,ID_PERSONA,edad,sexo,ecivil,e6a,oficio4_08,o10,o15,o18,o19,...,y3b_preg,y3e_preg,y4b_preg,y5a_preg,y5b_preg,s5,s13,s16,s28,r3
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,...,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,500.521935,43.384702,1.127109,3.575928,9.583802,6843.700787,44.665917,4.965129,1.386952,1.227222,...,1.976378,1.977503,1.853768,1.852643,1.965129,1.647919,1.129359,4.534308,18.686164,10.022497
std,289.461250,13.285772,0.333283,3.011451,2.191817,2134.611773,7.026637,0.212023,0.718865,0.633257,...,0.151954,0.148377,0.353537,0.354661,0.183555,1.288724,0.382800,1.144515,7.270779,2.424432
min,1.000000,16.000000,1.000000,1.000000,1.000000,1211.000000,5.000000,3.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,255.000000,32.000000,1.000000,1.000000,9.000000,6112.000000,45.000000,5.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,5.000000,22.000000,11.000000
50%,502.000000,43.000000,1.000000,2.000000,9.000000,7522.000000,45.000000,5.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,5.000000,22.000000,11.000000
75%,746.000000,54.000000,1.000000,8.000000,11.000000,8331.000000,45.000000,5.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,5.000000,22.000000,11.000000
max,1000.000000,79.000000,2.000000,8.000000,14.000000,9624.000000,84.000000,5.000000,3.000000,3.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,7.000000,4.000000,5.000000,22.000000,11.000000


# Feature Importance

### Funcion para determinar variables usando machine learning (RF)

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

def select_top_variables_rf(data, target_variable, top_n, id='ID_PERSONA'):

   # Remove rows with missing values
    final_df = data.dropna()

    # Extract independent variables (features) and dependent variable
    X = final_df.drop(target_variable, axis=1)
    y = final_df[target_variable]


    # remove ID persona column
    X = X.drop(id, axis=1, errors='ignore')

    # Initialize Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Fit the model
    rf_model.fit(X, y)

    # Obtain feature importances
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf_model.feature_importances_})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Select the top N variables based on importance values
    top_variables = feature_importances.head(top_n)['Feature'].tolist()

    return top_variables, feature_importances

# # Ejemplo de aplicacion
# selected_variables_rf = select_top_variables_rf(clean_data, target_variable='y1', top_n=6)
# print("Selected top variables (Random Forest):", selected_variables_rf)


In [9]:
selected_variables_rf, feature_importances = select_top_variables_rf(result, target_variable='y1', top_n=7)
print("Selected top variables (Random Forest):", selected_variables_rf)

Selected top variables (Random Forest): ['oficio4_08', 's13', 'edad', 'e6a', 's5', 'ecivil', 'o10']


**Importancia**

Las variables que más inciden en la brecha salarial son **oficio, , previsión de salud (s13), edad, nivel educacional (e6a), nro. de Hijos o hijas (s5), estado civil y horas de trabajo semanal (010)**.

Particularmente la variable s13 pareciera más bien una consecuencia del salario y no una variable que incida realmente, ya que las personas con mayor salario prefieren el sistema de isapres. Por otro lado, las horas por semana ciertamente impactan en el salario final, sin embargo, más del 80% de las personas tienen un jornada entre 37 y 50 horas.




In [10]:
feature_importances

,Feature,Importance
4,oficio4_08,0.402564
19,s13,0.149881
0,edad,0.111108
3,e6a,0.056833
18,s5,0.045844
2,ecivil,0.036464
5,o10,0.036288
1,sexo,0.021271
15,y4b_preg,0.019221
20,s16,0.014979


Variables importantes en hombres

In [11]:
man_data = result[result['sexo'] == 1]
selected_variables_rf, feature_importances = select_top_variables_rf(man_data, target_variable='y1', top_n=7)
print("Selected top variables (Random Forest):", selected_variables_rf)

Selected top variables (Random Forest): ['oficio4_08', 's13', 'edad', 'e6a', 'y4b_preg', 's5', 'o10']


In [12]:
feature_importances

,Feature,Importance
4,oficio4_08,0.427964
19,s13,0.142422
0,edad,0.093266
3,e6a,0.061808
15,y4b_preg,0.044353
18,s5,0.038024
5,o10,0.032909
2,ecivil,0.028778
11,o31,0.020295
20,s16,0.017447


Variables importantes en mujeres

In [13]:
woman_data = result[result['sexo'] == 2]
selected_variables_rf, feature_importances = select_top_variables_rf(woman_data, target_variable='y1', top_n=7)
print("Selected top variables (Random Forest):", selected_variables_rf)

Selected top variables (Random Forest): ['s13', 'oficio4_08', 'edad', 'o10', 'e6a', 'o19', 's5']


Es interesante que la mayor parte de las variables relevantes se mantienen a la hora de calcular el salario de hombres y mujeres, sin embargo el orden de importancia se altera. Por otro lado en ambos casos hay una varible diferente, para el caso de los **hombres** el indcador de **la gratifiación (y4b_preg)** y en el caso de las **mujeres** **el contrato de trabajo (019)**, lo cual puede interpretarse como 2 vistas diferentes del mismo fenómeno: la relación contractual y beneficios asociados.

In [14]:
feature_importances

,Feature,Importance
19,s13,0.542944
4,oficio4_08,0.178643
0,edad,0.063346
5,o10,0.050349
3,e6a,0.045717
8,o19,0.027777
18,s5,0.020304
16,y5a_preg,0.015704
2,ecivil,0.012841
21,s28,0.007476


# Gap

## the gap is calculated over all database

In [15]:
# man is 1
# woman is 2

women_income = data[data['sexo'] == 2]['y1']
men_income = data[data['sexo'] == 1]['y1']
avg_women_income = women_income.mean()
avg_men_income = men_income.mean()
gap = (avg_men_income - avg_women_income)/avg_men_income
gap

0.06667776329513446